In [60]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds


In [61]:
df_recipe = pd.read_csv('recipe_list.csv')
df_rating = pd.read_csv('user_rating.csv', encoding='cp949')
df_recipe.head()

,레시피id,사진,요리,재료,조리법,카테고리,Unnamed: 6
0,1,NaN,스팸 김밥,스팸 1통\n밥 2그릇\n달걀 4개\n맛소금\n김밥용 김 2장,"1. 밥에 참기름과 맛소금을 넣고 잘 버무려 주세요. 밥 두공기에, 맛소금 1/2 ...",NaN,"일식, 밥, 김밥, 분식, 돼지고기"
1,2,NaN,떡볶이,쌀떡 400g \n사각어묵 3장 \n비엔나 소세지 3개 \n삶은 달걀 2개 \n대파...,1. 떡은 미리 찬물에 2-3번 헹군 다음에 이렇게 담가두세요. 다른 재료를 준비하...,"분식, 떡, 매콤",NaN
2,3,NaN,핫도그,모닝빵 \n프랑크소세지 2개\n슬라이스 치즈 1장\n양파 반개\n양배추 채 적당히 ...,1. 우선 양파는 얇게 썰어줍니다\n2. 채썬 양파는 물에 담궈서 매운맛을 빼줍니다...,"양식, 간단, 샌드위치, 돼지고기",NaN
3,4,NaN,김치찌개,돼지고기 삼겹살 300g\n신김치 송송썰어 3컵 분량 (약 300g)\n쌀뜨물 3컵...,1. 오늘 육수는 쌀뜨물을 약 3~4컵 분량 준비해 둡니다. 쌀뜨물 대신 멸치육수를...,"한식, 매콤, 고기, 국, 김치, 돼지고기",NaN
4,5,NaN,김치말이국수,소면 100g \n잘익은 김치 80g \n삶은 계란 1개 \n오이 1/6개\n김치국...,1. 김치는 신걸로 준비해야 맛있습니다 양념을 털어내고 잘게 썰어주세요 설탕 0.3...,"한식, 매콤, 김치, 면, 분식",NaN


In [62]:
df_rating.head()

,사용자id,레시피id,평점,시간
0,1,1,2.5,1260759144
1,1,2,3.0,1260759179
2,1,3,3.0,1260759182
3,1,4,2.0,1260759185
4,1,5,4.0,1260759205


In [63]:
recipe_rating_matrix = df_rating.pivot(
    index='사용자id',
    columns='레시피id',
    values='평점'
).fillna(0)

In [64]:
matrix = recipe_rating_matrix.values
user_ratings_mean = np.mean(matrix, axis = 1)
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

In [65]:
pd.DataFrame(matrix_user_mean, columns = recipe_rating_matrix.columns).head()

레시피id,1,2,3,4,5,6,7,8,9,10,...,43,44,45,46,47,48,49,50,51,52
0,2.105769,2.605769,2.605769,1.605769,3.605769,1.605769,1.605769,1.605769,-0.394231,-0.394231,...,-0.394231,-0.394231,-0.394231,-0.394231,-0.394231,-0.394231,-0.394231,-0.394231,-0.394231,-0.394231
1,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846,3.346154,1.846154,...,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846,-0.153846
2,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,...,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769,-0.230769
3,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,...,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462
4,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,...,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462,-0.038462


In [66]:
# movie_user_rating = pod_value, user_movie_rating =pod_rating, 
U, sigma, Vt = svds(matrix_user_mean, k = 2)

In [67]:
sigma = np.diag(sigma)
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = recipe_rating_matrix.columns)

In [99]:
def print_predict(dataset):
    # 해당 column의 이름을 정해서 삭제 하고 싶은 부분을 조절 가능
    dataset = dataset.drop(['Unnamed: 6','Predictions'], axis=1)
    return dataset

In [104]:
def print_predict_list(dataset):
    # 위의 출력부분과 마찬가지의 파트
    dataset = dataset.drop(['재료','조리법'], axis=1)
    return dataset

In [103]:
def recommend_movies(df_svd_preds, user_id, recipe_df, rating_df, num_recommendations=3):
    
    user_row_number = user_id - 1
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    user_data = rating_df[rating_df.사용자id == user_id]
    
    user_history = user_data.merge(recipe_df, on = '레시피id').sort_values(['평점'], ascending=False)
    recommendations = recipe_df[~recipe_df['레시피id'].isin(user_history['레시피id'])]
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = '레시피id')
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
    
    recommendations = print_predict(recommendations)                  
    recommendations_list = print_predict_list(recommendations) 

    return recommendations_list, recommendations

In [107]:
# df_user_movie_ratings = recipe_rating_matrix
recommendations_list, recommendations = recommend_movies(df_svd_preds, 10, df_recipe, df_rating)

In [108]:
recommendations_list


,레시피id,사진,요리,카테고리
16,17,NaN,순두부 열라면,"한식, 라면, 분식, 면, 매콤"
15,16,NaN,옥수수 콘치즈,"양식, 치즈"
23,24,NaN,오코노미야키,"일식, 전"
